In [1]:
import os

os.chdir("/root/dev/human/")
os.getcwd()

'/root/dev/human'

In [2]:
import torch

torch.cuda.set_device(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!gpustat

045da8083fa8                Mon Oct  7 23:57:47 2024  550.100
[0] NVIDIA GeForce RTX 4090 | 37°C,   0 % |   396 / 24564 MB |


# Triton example

In [9]:
import triton
import triton.language as tl

@triton.jit
def matmul_kernel(
    A_ptr, B_ptr, C_ptr,
    M, N, K,
    stride_am, stride_ak,
    stride_bk, stride_bn,
    stride_cm, stride_cn,
    BLOCK_SIZE_M: tl.constexpr, BLOCK_SIZE_N: tl.constexpr, BLOCK_SIZE_K: tl.constexpr
):
    # Program ID identifies which block of the output matrix this kernel computes
    pid_m = tl.program_id(0)
    pid_n = tl.program_id(1)
    
    # Compute the starting indices of the block
    offs_m = pid_m * BLOCK_SIZE_M + tl.arange(0, BLOCK_SIZE_M)
    offs_n = pid_n * BLOCK_SIZE_N + tl.arange(0, BLOCK_SIZE_N)
    offs_k = tl.arange(0, BLOCK_SIZE_K)
    
    # Initialize accumulation to zero
    acc = tl.zeros((BLOCK_SIZE_M, BLOCK_SIZE_N), dtype=tl.float32)
    
    # Loop over K dimension
    for k in range(0, K, BLOCK_SIZE_K):
        # Load blocks from A and B
        a_ptrs = A_ptr + (offs_m[:, None] * stride_am + (k + offs_k[None, :]) * stride_ak)
        b_ptrs = B_ptr + ((k + offs_k[:, None]) * stride_bk + offs_n[None, :] * stride_bn)

        a = tl.load(a_ptrs, mask=(offs_m[:, None] < M) & (k + offs_k[None, :] < K), other=0.0)
        b = tl.load(b_ptrs, mask=(k + offs_k[:, None] < K) & (offs_n[None, :] < N), other=0.0)

        # Compute partial products and accumulate
        acc += tl.dot(a, b)

    # Write back to C
    c_ptrs = C_ptr + (offs_m[:, None] * stride_cm + offs_n[None, :] * stride_cn)
    tl.store(c_ptrs, acc, mask=(offs_m[:, None] < M) & (offs_n[None, :] < N))

In [10]:
import torch

# Define matrix sizes
M, N, K = 1024, 1024, 1024

# Create random matrices
A = torch.randn(M, K, device="cuda")
B = torch.randn(K, N, device="cuda")
C = torch.randn(M, N, device="cuda")

# Define block sizes
BLOCK_SIZE_M = 128
BLOCK_SIZE_N = 128
BLOCK_SIZE_K = 64

# Launch the kernel
grid = ( (M + BLOCK_SIZE_M - 1) // BLOCK_SIZE_M, (N + BLOCK_SIZE_N - 1) // BLOCK_SIZE_N )
matmul_kernel[grid](
    A, B, C,
    M, N, K,
    A.stride(0), A.stride(1),
    B.stride(0), B.stride(1),
    C.stride(0), C.stride(1),
    BLOCK_SIZE_M=BLOCK_SIZE_M, BLOCK_SIZE_N=BLOCK_SIZE_N, BLOCK_SIZE_K=BLOCK_SIZE_K,
)

OutOfResources: out of resource: shared memory, Required: 131072, Hardware limit: 101376. Reducing block sizes or `num_stages` may help.